In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm

# Assume 'mqy.csv' is loaded into pandas DataFrame 'df'

# --- Data Preparation ---
# Check if log variables exist, create if necessary (handle log(0))
# Example: if 'lgrain_pred' not in df.columns and 'grain_pred' in df.columns:
#     df['lgrain_pred'] = np.log(df['grain_pred'].replace(0, np.nan)) # Add small constant if needed instead of NaN
# Perform similar checks/creations for ldeath_b, lurbpop, ltotpop

# Create the interaction term
# Ensure 'famdum5860' exists - this dummy should be 1 for 1958, 1959, 1960, and 0 otherwise.
df['lgrain_pred_famdum5860'] = df['lgrain_pred'] * df['famdum5860']

# Define columns needed for regression
reg_cols = ['ldeath_b', 'lgrain_pred_famdum5860', 'lgrain_pred', 'lurbpop', 'ltotpop', 'prov', 'year']

# Drop rows with missing values in necessary columns
df_reg = df.dropna(subset=reg_cols).copy()

# --- Regression Estimation ---
formula = 'ldeath_b ~ lgrain_pred_famdum5860 + lgrain_pred + lurbpop + ltotpop + C(year)'
model = smf.ols(formula, data=df_reg)
results = model.fit()

# --- Get Coefficients and Standard Errors ---
coeffs = results.params
se_hc1 = results.get_robustcov_results(cov_type='HC1').bse

# Cluster SEs require the cluster variable(s)
# Ensure 'prov' is the correct province identifier column name
se_cluster_prov = results.get_robustcov_results(cov_type='cluster', groups=df_reg['prov']).bse

# Two-way cluster SEs (requires statsmodels >= 0.13 potentially)
# Ensure 'prov' and 'year' are the correct identifiers
try:
    se_cluster_prov_year = results.get_robustcov_results(cov_type='cluster', groups=df_reg[['prov', 'year']]).bse
    two_way_supported = True
except Exception as e:
    print(f"Could not compute two-way clustered SEs. Error: {e}")
    se_cluster_prov_year = pd.Series([np.nan] * len(coeffs), index=coeffs.index) # Placeholder
    two_way_supported = False


# --- Format and Print Results ---
results_summary = pd.DataFrame({
    'Coefficient': coeffs,
    'SE (HC1)': se_hc1,
    'SE (Cluster Prov)': se_cluster_prov,
    'SE (Cluster Prov-Year)': se_cluster_prov_year
})

# Filter for main variables of interest
main_vars = ['lgrain_pred_famdum5860', 'lgrain_pred', 'lurbpop', 'ltotpop', 'Intercept']
results_summary_main = results_summary.loc[results_summary.index.isin(main_vars)]

print("\n--- Regression Results ---")
print(results_summary_main)

# Extract N and T for part d
N_provinces = df_reg['prov'].nunique()
T_years = df_reg['year'].nunique()
print(f"\nNumber of provinces (N): {N_provinces}")
print(f"Number of time periods (T): {T_years}")